In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Load datasets
demand_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/Passenger Journeys by Public Transport.csv")
demographic_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/Demographic.csv")
weather_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/weather.csv")

In [3]:
demand_data.head()

,STATISTIC,Statistic Label,TLIST(A1),Year,C03935V04687,Mode of Transport,C01198V01436,Weeks of the year,UNIT,VALUE
0,THA24C01,Passenger Journeys,2019,2019,10,Dublin Metro Bus,1,Week 01,Number,1987891.0
1,THA24C01,Passenger Journeys,2019,2019,10,Dublin Metro Bus,2,Week 02,Number,2709579.0
2,THA24C01,Passenger Journeys,2019,2019,10,Dublin Metro Bus,3,Week 03,Number,2784678.0
3,THA24C01,Passenger Journeys,2019,2019,10,Dublin Metro Bus,4,Week 04,Number,2858346.0
4,THA24C01,Passenger Journeys,2019,2019,10,Dublin Metro Bus,5,Week 05,Number,2924821.0


In [4]:
demographic_data.head()

,STATISTIC,Statistic Label,TLIST(A1),Year,C02199V02655,Sex,C02076V02508,Age Group,UNIT,VALUE
0,NTA105C01,Frequency of use of bus services - Weekly or m...,2021,2021,1,Male,350,18 - 24 years,%,NaN
1,NTA105C01,Frequency of use of bus services - Weekly or m...,2021,2021,1,Male,415,25 - 34 years,%,19.0
2,NTA105C01,Frequency of use of bus services - Weekly or m...,2021,2021,1,Male,465,35 - 44 years,%,10.0
3,NTA105C01,Frequency of use of bus services - Weekly or m...,2021,2021,1,Male,500,45 - 54 years,%,9.0
4,NTA105C01,Frequency of use of bus services - Weekly or m...,2021,2021,1,Male,535,55 - 64 years,%,7.0


In [5]:
weather_data.head()

,date,ind,rain,ind.1,maxt,ind.2,mint,gmin,soil
0,01-Jan-41,0,2.4,,,,,,
1,02-Jan-41,0,0.9,,,,,,
2,03-Jan-41,0,0.0,,,,,,
3,04-Jan-41,0,0.0,,,,,,
4,05-Jan-41,0,0.0,,,,,,


In [6]:
# Displaying basic information about the datasets
print("Demand Data Info:")
print(demand_data.info())

print("\nDemographic Data Info:")
print(demographic_data.info())

print("\nWeather Data Info:")
print(weather_data.info())

# Summary statistics
print("\nDemand Data Summary Statistics:")
print(demand_data.describe())

print("\nDemographic Data Summary Statistics:")
print(demographic_data.describe())

print("\nWeather Data Summary Statistics:")
print(weather_data.describe())

Demand Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060 entries, 0 to 1059
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   STATISTIC          1060 non-null   object 
 1   Statistic Label    1060 non-null   object 
 2   TLIST(A1)          1060 non-null   int64  
 3   Year               1060 non-null   int64  
 4   C03935V04687       1060 non-null   int64  
 5   Mode of Transport  1060 non-null   object 
 6   C01198V01436       1060 non-null   int64  
 7   Weeks of the year  1060 non-null   object 
 8   UNIT               1060 non-null   object 
 9   VALUE              868 non-null    float64
dtypes: float64(1), int64(4), object(5)
memory usage: 82.9+ KB
None

Demographic Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATI

In [7]:
demand_data.columns

Index(['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'Year', 'C03935V04687',
       'Mode of Transport', 'C01198V01436', 'Weeks of the year', 'UNIT',
       'VALUE'],
      dtype='object')

In [8]:
demographic_data.columns

Index(['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'Year', 'C02199V02655',
       'Sex', 'C02076V02508', 'Age Group', 'UNIT', 'VALUE'],
      dtype='object')

In [9]:
weather_data.columns

Index(['date', 'ind', 'rain', 'ind.1', 'maxt', 'ind.2', 'mint', 'gmin',
       'soil'],
      dtype='object')

# Preparing the first dataset: demand_data

In [10]:
# Renaming columns
demand_data.rename(columns={'Weeks of the year': 'Weeks', 'VALUE': 'Demand'}, inplace=True)

In [11]:
demand_data.drop(['STATISTIC','TLIST(A1)','UNIT'], axis = 1, inplace = True)

In [12]:
#converting year and weeks columns to date column 
demand_data['Date'] = demand_data['Year'].astype(str) + '-W' + demand_data['Weeks'].str.slice(start=5).astype(int).apply(lambda x: f'{x:02}')
demand_data['Date'] = pd.to_datetime(demand_data['Date'] + '-1', format='%Y-W%U-%w')
demand_data['Date'] = demand_data['Date'].dt.strftime('%d/%m/%Y')

In [13]:
desired_order = ['Year', 'Weeks', 'Date', 'Mode of Transport', 'Statistic Label', 'Demand', 'C03935V04687', 'C01198V01436']

demand_data = demand_data.reindex(columns=desired_order)

#Dealing with the missing values on the dataset
demand_data.interpolate(method='linear', inplace=True)

#formating the Deamnd column to make it more simpler 
demand_data['Demand'] = demand_data['Demand'].apply(lambda x: "{:,.0f}".format(x))

# remove commas on Demand
demand_data['Demand'] = demand_data['Demand'].str.replace(',', '').astype(int)

In [14]:
demand_data.drop(['C03935V04687','C01198V01436'], axis = 1, inplace = True)

In [15]:
demand_data.columns

Index(['Year', 'Weeks', 'Date', 'Mode of Transport', 'Statistic Label',
       'Demand'],
      dtype='object')

In [16]:
demand_data.head()

,Year,Weeks,Date,Mode of Transport,Statistic Label,Demand
0,2019,Week 01,07/01/2019,Dublin Metro Bus,Passenger Journeys,1987891
1,2019,Week 02,14/01/2019,Dublin Metro Bus,Passenger Journeys,2709579
2,2019,Week 03,21/01/2019,Dublin Metro Bus,Passenger Journeys,2784678
3,2019,Week 04,28/01/2019,Dublin Metro Bus,Passenger Journeys,2858346
4,2019,Week 05,04/02/2019,Dublin Metro Bus,Passenger Journeys,2924821


In [17]:
demand_data['Date'] = pd.to_datetime(demand_data['Date'], format='%d/%m/%Y')